# Лабораторная работа №3 (Проведение исследований с решающим деревом)

### Создание бейзлайна и оценка качества

Импорт необходимых библиотек.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

Подготовка данных.

In [2]:
path = 'drive/MyDrive/ai_data/'
heart = pd.read_csv(path + 'heart.csv')
heart.shape, heart.columns

((918, 12),
 Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
        'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
        'HeartDisease'],
       dtype='object'))

Посмотрим состав таблицы и наличие пропусков, а также баланс HeartDisease.

In [3]:
display(heart.head())
display(heart.info())
display(heart.describe())
print("Missing per column:\n", heart.isna().sum())
print(heart['HeartDisease'].value_counts(normalize=True))

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


None

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


Missing per column:
 Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64
HeartDisease
1    0.553377
0    0.446623
Name: proportion, dtype: float64


Выполним те же действия для второго датасета.

In [4]:
melbourne = pd.read_csv(path + 'melb_data.csv')
melbourne.shape, melbourne.columns

((13580, 21),
 Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
        'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
        'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
        'Longtitude', 'Regionname', 'Propertycount'],
       dtype='object'))

In [5]:
display(melbourne.head())
display(melbourne.info())
display(melbourne.describe())
print(melbourne.isna().sum())
print(melbourne['Price'].describe())

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

None

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


Suburb              0
Address             0
Rooms               0
Type                0
Price               0
Method              0
SellerG             0
Date                0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
CouncilArea      1369
Lattitude           0
Longtitude          0
Regionname          0
Propertycount       0
dtype: int64
count    1.358000e+04
mean     1.075684e+06
std      6.393107e+05
min      8.500000e+04
25%      6.500000e+05
50%      9.030000e+05
75%      1.330000e+06
max      9.000000e+06
Name: Price, dtype: float64


Удалим столбцы с большим количеством пропусков, заполним столбцы с маленьким количеством пропусков медианой и модой

In [6]:
columns_to_drop = ['BuildingArea', 'YearBuilt']
melbourne_clean = melbourne.drop(columns=columns_to_drop)

car_median = melbourne_clean['Car'].median()
melbourne_clean['Car'] = melbourne_clean['Car'].fillna(car_median)

council_mode = melbourne_clean['CouncilArea'].mode()[0]
melbourne_clean['CouncilArea'] = melbourne_clean['CouncilArea'].fillna(council_mode)

print(melbourne_clean.isna().sum())

Suburb           0
Address          0
Rooms            0
Type             0
Price            0
Method           0
SellerG          0
Date             0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Car              0
Landsize         0
CouncilArea      0
Lattitude        0
Longtitude       0
Regionname       0
Propertycount    0
dtype: int64


Разделим данные на тренировочную и тестовую выборки:

In [7]:
X_heart = heart.drop('HeartDisease', axis=1)
y_heart = heart['HeartDisease']
X_heart_train, X_heart_test, y_heart_train, y_heart_test = train_test_split(
    X_heart, y_heart, test_size=0.2, random_state=42, stratify=y_heart
)


X_melbourne = melbourne_clean.drop('Price', axis=1)
y_melbourne = melbourne_clean['Price']
X_melbourne_train, X_melbourne_test, y_melbourne_train, y_melbourne_test = train_test_split(
    X_melbourne, y_melbourne, test_size=0.2, random_state=42
)

Проведем обработку категориальных переменных:

In [8]:
X_heart_train_processed = X_heart_train.copy()
X_heart_test_processed = X_heart_test.copy()
X_melbourne_train_processed = X_melbourne_train.copy()
X_melbourne_test_processed = X_melbourne_test.copy()

Обработка категориальных переменных для Heart dataset:

In [9]:
categorical_heart = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
for col in categorical_heart:
    print(f"{col}: {X_heart_train[col].unique()}")

label_encoders_heart = {}
for col in categorical_heart:
    le = LabelEncoder()
    X_heart_train_processed[col] = le.fit_transform(X_heart_train[col])
    X_heart_test_processed[col] = le.transform(X_heart_test[col])
    label_encoders_heart[col] = le

print("\nРезультат:")
print(X_heart_train_processed[categorical_heart].head())
print(f"{X_heart_train_processed[categorical_heart].dtypes}")

Sex: ['M' 'F']
ChestPainType: ['ATA' 'ASY' 'NAP' 'TA']
RestingECG: ['ST' 'Normal' 'LVH']
ExerciseAngina: ['Y' 'N']
ST_Slope: ['Flat' 'Up' 'Down']

Результат:
     Sex  ChestPainType  RestingECG  ExerciseAngina  ST_Slope
485    1              1           2               1         1
486    1              1           2               0         2
117    0              0           2               1         1
361    1              0           1               1         1
296    1              0           1               1         1
Sex               int64
ChestPainType     int64
RestingECG        int64
ExerciseAngina    int64
ST_Slope          int64
dtype: object


Обработка категориальных переменных для Melbourne dataset. Применяем One-Hot Encoding для сложных категориальных признаков. Удаляем нечисловые столбцы, которые не будем использовать.

In [10]:
categorical_melbourne = ['Type', 'Method', 'CouncilArea', 'Regionname']
for col in categorical_melbourne:
    print(f"{col}: {X_melbourne_train[col].nunique()}")

simple_categorical = ['Type', 'Method']
complex_categorical = ['CouncilArea', 'Regionname']

label_encoders_melbourne = {}
for col in simple_categorical:
    le = LabelEncoder()
    X_melbourne_train_processed[col] = le.fit_transform(X_melbourne_train[col])
    X_melbourne_test_processed[col] = le.transform(X_melbourne_test[col])
    label_encoders_melbourne[col] = le


council_dummies_train = pd.get_dummies(X_melbourne_train['CouncilArea'], prefix='Council', dtype=int)
region_dummies_train = pd.get_dummies(X_melbourne_train['Regionname'], prefix='Region', dtype=int)
council_dummies_test = pd.get_dummies(X_melbourne_test['CouncilArea'], prefix='Council', dtype=int)
region_dummies_test = pd.get_dummies(X_melbourne_test['Regionname'], prefix='Region', dtype=int)

council_dummies_test = council_dummies_test.reindex(columns=council_dummies_train.columns, fill_value=0)
region_dummies_test = region_dummies_test.reindex(columns=region_dummies_train.columns, fill_value=0)

X_melbourne_train_processed = X_melbourne_train_processed.drop(['CouncilArea', 'Regionname'], axis=1)
X_melbourne_test_processed = X_melbourne_test_processed.drop(['CouncilArea', 'Regionname'], axis=1)

X_melbourne_train_processed = pd.concat([X_melbourne_train_processed, council_dummies_train, region_dummies_train], axis=1)
X_melbourne_test_processed = pd.concat([X_melbourne_test_processed, council_dummies_test, region_dummies_test], axis=1)

columns_to_drop_melbourne = ['Suburb', 'Address', 'SellerG', 'Date']
X_melbourne_train_processed = X_melbourne_train_processed.drop(columns=columns_to_drop_melbourne, errors='ignore')
X_melbourne_test_processed = X_melbourne_test_processed.drop(columns=columns_to_drop_melbourne, errors='ignore')

print(f"Итого: {X_melbourne_train_processed.shape[1]} признаков")

Type: 3
Method: 5
CouncilArea: 33
Regionname: 8
Итого: 53 признаков


Итоговые размер данных и типы данных:

In [11]:
print(f"Heart train: {X_heart_train_processed.shape}")
print(f"Heart test: {X_heart_test_processed.shape}")
print(f"Melbourne train: {X_melbourne_train_processed.shape}")
print(f"Melbourne test: {X_melbourne_test_processed.shape}")

print(f"Heart: {X_heart_train_processed.dtypes.unique()}")
print(f"Melbourne: {X_melbourne_train_processed.dtypes.unique()}")

Heart train: (734, 11)
Heart test: (184, 11)
Melbourne train: (10864, 53)
Melbourne test: (2716, 53)
Heart: [dtype('int64') dtype('float64')]
Melbourne: [dtype('int64') dtype('float64')]


Проведем масштабирование признаков:

In [12]:
scaler_heart = StandardScaler()
X_heart_train_scaled = scaler_heart.fit_transform(X_heart_train_processed)
X_heart_test_scaled = scaler_heart.transform(X_heart_test_processed)

scaler_melbourne = StandardScaler()
X_melbourne_train_scaled = scaler_melbourne.fit_transform(X_melbourne_train_processed)
X_melbourne_test_scaled = scaler_melbourne.transform(X_melbourne_test_processed)

#### Обучение моделей

In [13]:
tree_classifier = DecisionTreeClassifier(random_state=42)
tree_classifier.fit(X_heart_train_scaled, y_heart_train)

y_heart_pred = tree_classifier.predict(X_heart_test_scaled)

tree_regressor = DecisionTreeRegressor(random_state=42)
tree_regressor.fit(X_melbourne_train_scaled, y_melbourne_train)

y_melbourne_pred = tree_regressor.predict(X_melbourne_test_scaled)

#### Оценка качества моделей

In [14]:
print("Классификация")
accuracy = accuracy_score(y_heart_test, y_heart_pred)
recall = recall_score(y_heart_test, y_heart_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")

print("\nРегрессия")
r2 = r2_score(y_melbourne_test, y_melbourne_pred)
mae = mean_absolute_error(y_melbourne_test, y_melbourne_pred)
print(f"R2: {r2:.4f}")
print(f"MAE: {mae:,.2f}")

Классификация
Accuracy: 0.7880
Recall: 0.8039

Регрессия
R2: 0.6486
MAE: 225,051.93


### Улучшение бейзлайна

Гипотезы:
- Ограничение глубины дерева (max_depth) предотвратит переобучение и улучшит генерализацию модели.
- Увеличение минимального количества образцов для разбиения узла (min_samples_split) и листа (min_samples_leaf) сделает дерево более устойчивым к шуму в данных.
- Изменение критерия разбиения с 'gini' на 'entropy' для классификации и с 'squared_error' на 'absolute_error' для регрессии может улучшить качество предсказаний.



Гипотеза 1:

In [15]:
param_grid_depth = {
    'max_depth': [3, 5, 7, 10, 15, 20, None]
}

grid_class_depth = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid_depth,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)
grid_class_depth.fit(X_heart_train_scaled, y_heart_train)

grid_reg_depth = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid_depth,
    cv=5,
    scoring='r2',
    n_jobs=-1
)
grid_reg_depth.fit(X_melbourne_train_scaled, y_melbourne_train)

print(f"Классификация - Лучший max_depth: {grid_class_depth.best_params_['max_depth']}")
print(f"Лучшая accuracy на кросс-валидации: {grid_class_depth.best_score_:.4f}")

print(f"\nРегрессия - Лучший max_depth: {grid_reg_depth.best_params_['max_depth']}")
print(f"Лучший R2 на кросс-валидации: {grid_reg_depth.best_score_:.4f}")

Классификация - Лучший max_depth: 3
Лучшая accuracy на кросс-валидации: 0.8365

Регрессия - Лучший max_depth: 10
Лучший R2 на кросс-валидации: 0.6352


Гипотеза 1 подтвердилась только для классификации, для регрессии метрики ухудшились.  
Гипотеза 2:

In [16]:
param_grid_samples = {
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10]
}

grid_class_samples = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid_samples,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)
grid_class_samples.fit(X_heart_train_scaled, y_heart_train)

grid_reg_samples = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid_samples,
    cv=5,
    scoring='r2',
    n_jobs=-1
)
grid_reg_samples.fit(X_melbourne_train_scaled, y_melbourne_train)

print(f"Классификация - Лучшие параметры: {grid_class_samples.best_params_}")
print(f"Лучшая accuracy на кросс-валидации: {grid_class_samples.best_score_:.4f}")

print(f"\nРегрессия - Лучшие параметры: {grid_reg_samples.best_params_}")
print(f"Лучший R2 на кросс-валидации: {grid_reg_samples.best_score_:.4f}")

Классификация - Лучшие параметры: {'min_samples_leaf': 5, 'min_samples_split': 20}
Лучшая accuracy на кросс-валидации: 0.8310

Регрессия - Лучшие параметры: {'min_samples_leaf': 5, 'min_samples_split': 20}
Лучший R2 на кросс-валидации: 0.7015


Гипотеза 2 подтвердилась, метрики улучшились.  
Гипотеза 3:

In [17]:
param_grid_criterion_class = {
    'criterion': ['gini', 'entropy']
}

param_grid_criterion_reg = {
    'criterion': ['squared_error', 'absolute_error', 'friedman_mse']
}

grid_class_criterion = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid_criterion_class,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)
grid_class_criterion.fit(X_heart_train_scaled, y_heart_train)

grid_reg_criterion = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid_criterion_reg,
    cv=5,
    scoring='r2',
    n_jobs=-1
)
grid_reg_criterion.fit(X_melbourne_train_scaled, y_melbourne_train)

print(f"Классификация - Лучший criterion: {grid_class_criterion.best_params_['criterion']}")
print(f"Лучшая accuracy на кросс-валидации: {grid_class_criterion.best_score_:.4f}")

print(f"\nРегрессия - Лучший criterion: {grid_reg_criterion.best_params_['criterion']}")
print(f"Лучший R2 на кросс-валидации: {grid_reg_criterion.best_score_:.4f}")

Классификация - Лучший criterion: gini
Лучшая accuracy на кросс-валидации: 0.7983

Регрессия - Лучший criterion: squared_error
Лучший R2 на кросс-валидации: 0.5799


Гипотеза 3 подтвердилась только для классификации, для регрессии метрики ухудшились.  
Сформируем улучшенный бейзлайн по результатам проверки гипотез:

In [18]:
best_tree_classifier = DecisionTreeClassifier(
    max_depth=3,
    min_samples_split=20,
    min_samples_leaf=5,
    criterion='gini',
    random_state=42
)
best_tree_classifier.fit(X_heart_train_scaled, y_heart_train)

y_heart_pred_improved = best_tree_classifier.predict(X_heart_test_scaled)

best_tree_regressor = DecisionTreeRegressor(
    min_samples_split=20,
    min_samples_leaf=5,
    random_state=42
)
best_tree_regressor.fit(X_melbourne_train_scaled, y_melbourne_train)

y_melbourne_pred_improved = best_tree_regressor.predict(X_melbourne_test_scaled)

print("Классификация")
accuracy_improved = accuracy_score(y_heart_test, y_heart_pred_improved)
recall_improved = recall_score(y_heart_test, y_heart_pred_improved)
print(f"Accuracy: {accuracy_improved:.4f}")
print(f"Recall: {recall_improved:.4f}")

print("\nРегрессия")
r2_improved = r2_score(y_melbourne_test, y_melbourne_pred_improved)
mae_improved = mean_absolute_error(y_melbourne_test, y_melbourne_pred_improved)
print(f"R2: {r2_improved:.4f}")
print(f"MAE: {mae_improved:,.2f}")

print("\nСравнение с базовым бейзлайном:")
print(f"Классификация - Accuracy: {accuracy:.4f} -> {accuracy_improved:.4f}")
print(f"Классификация - Recall: {recall:.4f} -> {recall_improved:.4f}")
print(f"Регрессия - R2: {r2:.4f} -> {r2_improved:.4f}")
print(f"Регрессия - MAE: {mae:,.2f} -> {mae_improved:,.2f}")

Классификация
Accuracy: 0.8207
Recall: 0.7941

Регрессия
R2: 0.7478
MAE: 198,936.60

Сравнение с базовым бейзлайном:
Классификация - Accuracy: 0.7880 -> 0.8207
Классификация - Recall: 0.8039 -> 0.7941
Регрессия - R2: 0.6486 -> 0.7478
Регрессия - MAE: 225,051.93 -> 198,936.60


С улучшенным бейзлайном качество моделей улучшилось, поэтому можно сделать следующие выводы:
- Регуляризация через ограничение параметров дерева улучшила качество обеих моделей
- Для классификации эффективны все три подхода (глубина, размеры узлов, критерий)
- Для регрессии ключевую роль сыграли параметры min_samples_split и min_samples_leaf
- Улучшенный бейзлайн показал лучшие результаты по основным метрикам по сравнению с базовым

### Имплементация алгоритма машинного обучения

Релизация для регресии и классификации в виде классов:

In [19]:
class CustomDecisionTreeClassifier:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, criterion='gini'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.criterion = criterion
        self.tree = None

    def _gini(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)

    def _entropy(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))

    def _calculate_criterion(self, y):
        if self.criterion == 'gini':
            return self._gini(y)
        else:
            return self._entropy(y)

    def _split(self, X, y, feature, threshold):
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask
        return X[left_mask], X[right_mask], y[left_mask], y[right_mask]

    def _find_best_split(self, X, y):
        best_gain = -1
        best_feature = None
        best_threshold = None
        current_criterion = self._calculate_criterion(y)

        n_features = X.shape[1]
        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                X_left, X_right, y_left, y_right = self._split(X, y, feature, threshold)

                if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
                    continue

                n = len(y)
                n_left, n_right = len(y_left), len(y_right)
                criterion_left = self._calculate_criterion(y_left)
                criterion_right = self._calculate_criterion(y_right)

                gain = current_criterion - (n_left / n * criterion_left + n_right / n * criterion_right)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _build_tree(self, X, y, depth=0):
        n_samples = len(y)
        n_classes = len(np.unique(y))

        if (self.max_depth is not None and depth >= self.max_depth) or \
           n_classes == 1 or \
           n_samples < self.min_samples_split:
            leaf_value = np.bincount(y).argmax()
            return {'leaf': True, 'value': leaf_value}

        feature, threshold = self._find_best_split(X, y)

        if feature is None:
            leaf_value = np.bincount(y).argmax()
            return {'leaf': True, 'value': leaf_value}

        X_left, X_right, y_left, y_right = self._split(X, y, feature, threshold)

        left_subtree = self._build_tree(X_left, y_left, depth + 1)
        right_subtree = self._build_tree(X_right, y_right, depth + 1)

        return {
            'leaf': False,
            'feature': feature,
            'threshold': threshold,
            'left': left_subtree,
            'right': right_subtree
        }

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.tree = self._build_tree(X, y)
        return self

    def _predict_sample(self, x, tree):
        if tree['leaf']:
            return tree['value']

        if x[tree['feature']] <= tree['threshold']:
            return self._predict_sample(x, tree['left'])
        else:
            return self._predict_sample(x, tree['right'])

    def predict(self, X):
        X = np.array(X)
        return np.array([self._predict_sample(x, self.tree) for x in X])


class CustomDecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, criterion='squared_error'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.criterion = criterion
        self.tree = None

    def _mse(self, y):
        if len(y) == 0:
            return 0
        return np.mean((y - np.mean(y)) ** 2)

    def _mae(self, y):
        if len(y) == 0:
            return 0
        return np.mean(np.abs(y - np.mean(y)))

    def _calculate_criterion(self, y):
        if self.criterion == 'squared_error':
            return self._mse(y)
        else:
            return self._mae(y)

    def _split(self, X, y, feature, threshold):
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask
        return X[left_mask], X[right_mask], y[left_mask], y[right_mask]

    def _find_best_split(self, X, y):
        best_gain = -1
        best_feature = None
        best_threshold = None
        current_criterion = self._calculate_criterion(y)

        n_features = X.shape[1]
        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                X_left, X_right, y_left, y_right = self._split(X, y, feature, threshold)

                if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
                    continue

                n = len(y)
                n_left, n_right = len(y_left), len(y_right)
                criterion_left = self._calculate_criterion(y_left)
                criterion_right = self._calculate_criterion(y_right)

                gain = current_criterion - (n_left / n * criterion_left + n_right / n * criterion_right)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    def _build_tree(self, X, y, depth=0):
        n_samples = len(y)

        if (self.max_depth is not None and depth >= self.max_depth) or \
           n_samples < self.min_samples_split:
            leaf_value = np.mean(y)
            return {'leaf': True, 'value': leaf_value}

        feature, threshold = self._find_best_split(X, y)

        if feature is None:
            leaf_value = np.mean(y)
            return {'leaf': True, 'value': leaf_value}

        X_left, X_right, y_left, y_right = self._split(X, y, feature, threshold)

        left_subtree = self._build_tree(X_left, y_left, depth + 1)
        right_subtree = self._build_tree(X_right, y_right, depth + 1)

        return {
            'leaf': False,
            'feature': feature,
            'threshold': threshold,
            'left': left_subtree,
            'right': right_subtree
        }

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)
        self.tree = self._build_tree(X, y)
        return self

    def _predict_sample(self, x, tree):
        if tree['leaf']:
            return tree['value']

        if x[tree['feature']] <= tree['threshold']:
            return self._predict_sample(x, tree['left'])
        else:
            return self._predict_sample(x, tree['right'])

    def predict(self, X):
        X = np.array(X)
        return np.array([self._predict_sample(x, self.tree) for x in X])

Посмотрим на результаты и сравним их с базовым бейзайном:

In [20]:
custom_tree_classifier = CustomDecisionTreeClassifier()
custom_tree_classifier.fit(X_heart_train_scaled, y_heart_train)
y_heart_pred_custom = custom_tree_classifier.predict(X_heart_test_scaled)

custom_tree_regressor = CustomDecisionTreeRegressor()
custom_tree_regressor.fit(X_melbourne_train_scaled, y_melbourne_train)
y_melbourne_pred_custom = custom_tree_regressor.predict(X_melbourne_test_scaled)

accuracy_custom = accuracy_score(y_heart_test, y_heart_pred_custom)
recall_custom = recall_score(y_heart_test, y_heart_pred_custom)

r2_custom = r2_score(y_melbourne_test, y_melbourne_pred_custom)
mae_custom = mean_absolute_error(y_melbourne_test, y_melbourne_pred_custom)

print(f"Классификация - Accuracy: {accuracy_custom:.4f}, Recall: {recall_custom:.4f}")
print(f"Регрессия - R2: {r2_custom:.4f}, MAE: {mae_custom:,.2f}")

print("\nСравнение с базовым бейзлайном:")
print(f"Классификация - Sklearn Accuracy: {accuracy:.4f}, Custom Accuracy: {accuracy_custom:.4f}")
print(f"Регрессия - Sklearn R2: {r2:.4f}, Custom R2: {r2_custom:.4f}")

Классификация - Accuracy: 0.7663, Recall: 0.8039
Регрессия - R2: 0.6693, MAE: 220,878.75

Сравнение с базовым бейзлайном:
Классификация - Sklearn Accuracy: 0.7880, Custom Accuracy: 0.7663
Регрессия - Sklearn R2: 0.6486, Custom R2: 0.6693


Получили похожий на обычный бейзлайн результат. Отличие составляет несколько процентов, можно объяснить особенностями выбора порогов разбиения и критериев остановки.  
Теперь добавим в модель техники из улучшенного бейзлайна:

In [21]:
custom_tree_classifier_improved = CustomDecisionTreeClassifier(
    max_depth=3,
    min_samples_split=20,
    min_samples_leaf=5,
    criterion='gini'
)
custom_tree_classifier_improved.fit(X_heart_train_scaled, y_heart_train)
y_heart_pred_custom_improved = custom_tree_classifier_improved.predict(X_heart_test_scaled)

custom_tree_regressor_improved = CustomDecisionTreeRegressor(
    min_samples_split=20,
    min_samples_leaf=5
)
custom_tree_regressor_improved.fit(X_melbourne_train_scaled, y_melbourne_train)
y_melbourne_pred_custom_improved = custom_tree_regressor_improved.predict(X_melbourne_test_scaled)

accuracy_custom_improved = accuracy_score(y_heart_test, y_heart_pred_custom_improved)
recall_custom_improved = recall_score(y_heart_test, y_heart_pred_custom_improved)

r2_custom_improved = r2_score(y_melbourne_test, y_melbourne_pred_custom_improved)
mae_custom_improved = mean_absolute_error(y_melbourne_test, y_melbourne_pred_custom_improved)

print(f"Классификация - Accuracy: {accuracy_custom_improved:.4f}, Recall: {recall_custom_improved:.4f}")
print(f"Регрессия - R2: {r2_custom_improved:.4f}, MAE: {mae_custom_improved:,.2f}")

print("\nСравнение с улучшенным бейзлайном sklearn:")
print(f"Классификация - Sklearn Accuracy: {accuracy_improved:.4f}, Custom Accuracy: {accuracy_custom_improved:.4f}")
print(f"Классификация - Sklearn Recall: {recall_improved:.4f}, Custom Recall: {recall_custom_improved:.4f}")
print(f"Регрессия - Sklearn R2: {r2_improved:.4f}, Custom R2: {r2_custom_improved:.4f}")
print(f"Регрессия - Sklearn MAE: {mae_improved:,.2f}, Custom MAE: {mae_custom_improved:,.2f}")

Классификация - Accuracy: 0.8207, Recall: 0.7941
Регрессия - R2: 0.7485, MAE: 198,771.72

Сравнение с улучшенным бейзлайном sklearn:
Классификация - Sklearn Accuracy: 0.8207, Custom Accuracy: 0.8207
Классификация - Sklearn Recall: 0.7941, Custom Recall: 0.7941
Регрессия - Sklearn R2: 0.7478, Custom R2: 0.7485
Регрессия - Sklearn MAE: 198,936.60, Custom MAE: 198,771.72


Получили практически идентичный результат.

### Выводы

- Decision Tree эффективен для обеих задач при правильном подборе гиперпараметров
- Ограничение глубины и размеров узлов критично для предотвращения переобучения
- Собственная реализация алгоритма работает корректно и показывает идентичный результат
- Улучшенные модели показали существенный прирост качества по всем основным метрикам